In [16]:
!pip install datasets stanza

In [17]:
from datasets import load_dataset

dataset = load_dataset("Overfit-GM/turkish-toxic-language")

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'target', 'source', 'is_toxic'],
        num_rows: 77800
    })
})

In [18]:
# ignore unnecessary warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# numpy and pandas for process and analyze
import pandas as pd
import numpy as np

from nltk.corpus import stopwords
import nltk
import re

# stanza for NLP preprocessing
!pip install stanza
import stanza

# sklearn libraries for models and evaulation metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)

# matpilotlib and seaborn libraries for visualize
import matplotlib.pyplot as plt
import seaborn as sns

# progress bar for data processing
from tqdm import tqdm

# install and download stanza's turkish model
stanza.download("tr", verbose=False)
print("Stanza Turkish model downloaded!")

nlp = stanza.Pipeline("tr", use_gpu=True)

print("Libraries and NLTK datasets loaded!")

Stanza Turkish model downloaded!
Libraries and NLTK datasets loaded!


In [19]:
nltk.download("stopwords")
stop_words = set(stopwords.words("turkish"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
# function: remove special char
def remove_special_characters(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'[\U00010000-\U0010FFFF]', '', text)
    return text

# preprocessing function
def preprocess_text_stanza(text):
    text = remove_special_characters(text.lower())

    doc = nlp(text)
    lemmatized_tokens = [
        word.lemma if word.lemma is not None else word.text
        for sentence in doc.sentences for word in sentence.words
        if word.text.isalpha() and word.text not in stop_words
    ]
    return " ".join(lemmatized_tokens)

tqdm.pandas()

train = dataset['train'].select(range(50))
train = train.to_pandas()
train["cleaned_text"] = train["text"].progress_apply(preprocess_text_stanza)

train

100%|██████████| 50/50 [01:04<00:00,  1.28s/it]


,text,target,source,is_toxic,cleaned_text
0,Phil Spector bir lanet katil olduğunu Biliyors...,INSULT,jigsaw,1,phil spector bir lanet katil ol bil ben tıpkı ...
1,Lan siktirin gidin AMK pozitif sik kafaları Ül...,PROFANITY,told,1,la sik git amk pozitif sik kafa ülke ana dikil...
2,2pac Olmak İstiyorum Ja Rule bir Tupac Shakur ...,OTHER,jigsaw,0,ol iste ja rul bir tupac shakur ol iste çoğu z...
3,Ne yapılması gerekiyor Aradan sonra bu sayfaya...,OTHER,jigsaw,0,yap gerek ara sonra sayfa bak gör kölelik din ...
4,Kabul Dream Chaser programı ile ilgili olmayan...,OTHER,jigsaw,0,kabul dream chaser program ilgili ol çok kal m
5,Thanx Teşekkürler Bay horoz Bana son bir şans ...,OTHER,jigsaw,0,thanx teşekkür bay horoz ben son bir şans ver ...
6,yarının başında istatistikleri alt üst eden Er...,PROFANITY,told,1,yarı baş istatistik alt üst et ersun hoca fene...
7,Kim söyledi bunu bana Yıkıcı yaptığım şeylerde...,INSULT,jigsaw,1,söyle bu ben yık ci yap şey herhangi kişi bot ...
8,amını yolunu siktimin çocuğu support beğenmiyo...,PROFANITY,told,1,am yol sik çocuk support beğen adc sorak al an...
9,Zeka yaşı falan da orası ayrı sjsjd,INSULT,told,1,zeka yaşı falan ora ayrı sjsjd
